Użyj każdą z tych funkcji 
* `unix_timestamp()` 
* `date_format()`
* `to_unix_timestamp()`
* `from_unixtime()`
* `to_date()` 
* `to_timestamp()` 
* `from_utc_timestamp()` 
* `to_utc_timestamp()`

In [0]:
%scala
import org.apache.spark.sql.functions._

val kolumny = Seq("timestamp","unix", "Date")
val dane = Seq(("2015-03-22T14:13:34", 1646641525847L,"May, 2021"),
               ("2015-03-22T15:03:18", 1646641557555L,"Mar, 2021"),
               ("2015-03-22T14:38:39", 1646641578622L,"Jan, 2021"))

var dataFrame = spark.createDataFrame(dane).toDF(kolumny:_*)
  .withColumn("current_date",current_date().as("current_date"))
  .withColumn("current_timestamp",current_timestamp().as("current_timestamp"))
display(dataFrame)

timestamp,unix,Date,current_date,current_timestamp
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T10:49:55.356+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T10:49:55.356+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T10:49:55.356+0000


In [0]:
%scala
dataFrame.printSchema()

root
-- timestamp: string (nullable = true)
-- unix: long (nullable = false)
-- Date: string (nullable = true)
-- current_date: date (nullable = false)
-- current_timestamp: timestamp (nullable = false)

## unix_timestamp(..) & cast(..)

Konwersja **string** to a **timestamp**.

Lokalizacja funkcji 
* `pyspark.sql.functions` in the case of Python
* `org.apache.spark.sql.functions` in the case of Scala & Java

## 1. Zmiana formatu wartości timestamp yyyy-MM-dd'T'HH:mm:ss 
`unix_timestamp(..)`

Dokumentacja API `unix_timestamp(..)`:
> Convert time string with given pattern (see <a href="http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html" target="_blank">SimpleDateFormat</a>) to Unix time stamp (in seconds), return null if fail.

`SimpleDataFormat` is part of the Java API and provides support for parsing and formatting date and time values.

In [0]:
%scala
import org.apache.spark.sql.functions._

val zmianaFormatu = dataFrame
  .withColumn("unix_timestamp", unix_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss"))
  .withColumn("formatted_date", from_unixtime(col("unix_timestamp"), "yyyy-MM-dd HH:mm:ss"))
display(zmianaFormatu)

timestamp,unix,Date,current_date,current_timestamp,unix_timestamp,formatted_date
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T10:50:50.122+0000,1427033614,2015-03-22 14:13:34
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T10:50:50.122+0000,1427036598,2015-03-22 15:03:18
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T10:50:50.122+0000,1427035119,2015-03-22 14:38:39


2. Zmień format zgodnie z klasą `SimpleDateFormat`**yyyy-MM-dd HH:mm:ss**
  * a. Wyświetl schemat i dane żeby sprawdzicz czy wartości się zmieniły

In [0]:

%scala
val zmianaFormatu = dataFrame.withColumn("formatted_timestamp", date_format(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))

zmianaFormatu.printSchema()

root
-- timestamp: string (nullable = true)
-- unix: long (nullable = false)
-- Date: string (nullable = true)
-- current_date: date (nullable = false)
-- current_timestamp: timestamp (nullable = false)
-- formatted_timestamp: string (nullable = true)

zmianaFormatu: org.apache.spark.sql.DataFrame = [timestamp: string, unix: bigint ... 4 more fields]

In [0]:
%scala
//unix_timestamp
val tempE = dataFrame.withColumn("unix_timestamp", unix_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss"))
display(tempE)

timestamp,unix,Date,current_date,current_timestamp,unix_timestamp
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T10:57:35.945+0000,1427033614
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T10:57:35.945+0000,1427036598
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T10:57:35.945+0000,1427035119


## Stwórz nowe kolumny do DataFrame z wartościami year(..), month(..), dayofyear(..)

In [0]:
%scala
//date_format
val yearDate = dataFrame
  .withColumn("year", year(col("timestamp")))
  .withColumn("month", month(col("timestamp")))
  .withColumn("day_of_year", dayofyear(col("timestamp")))

display(yearDate)

timestamp,unix,Date,current_date,current_timestamp,year,month,day_of_year
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T10:58:15.793+0000,2015,3,81
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T10:58:15.793+0000,2015,3,81
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T10:58:15.793+0000,2015,3,81


In [0]:
%scala
//to_date()
val toDate = dataFrame.withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss"))
display(toDate)

timestamp,unix,date,current_date,current_timestamp
2015-03-22T14:13:34,1646641525847,2015-03-22,2025-03-22,2025-03-22T10:58:37.514+0000
2015-03-22T15:03:18,1646641557555,2015-03-22,2025-03-22,2025-03-22T10:58:37.514+0000
2015-03-22T14:38:39,1646641578622,2015-03-22,2025-03-22,2025-03-22T10:58:37.514+0000


In [0]:
%scala
//from_unixtime()
val fromUnix = dataFrame.withColumn("formatted_date", from_unixtime(unix_timestamp(), "yyyy-MM-dd HH:mm:ss"))
display(fromUnix)

timestamp,unix,Date,current_date,current_timestamp,formatted_date
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T11:00:03.844+0000,2025-03-22 11:00:03
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T11:00:03.844+0000,2025-03-22 11:00:03
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T11:00:03.844+0000,2025-03-22 11:00:03


In [0]:
%scala
//to_timestamp()
val toTimestamp = dataFrame.withColumn("new", to_timestamp($"timestamp", "yyyy-MM-dd'T'HH:mm:ss"))
display(toTimestamp)


timestamp,unix,Date,current_date,current_timestamp,new
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T11:00:37.932+0000,2015-03-22T14:13:34.000+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T11:00:37.932+0000,2015-03-22T15:03:18.000+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T11:00:37.932+0000,2015-03-22T14:38:39.000+0000


In [0]:
%scala
//to_utc_timestamp()
val toUtcTimestamp = dataFrame.withColumn("UTC", to_utc_timestamp(to_timestamp($"timestamp", "yyyy-MM-dd'T'HH:mm:ss"), "Europe/Warsaw"))
display(toUtcTimestamp)



timestamp,unix,Date,current_date,current_timestamp,UTC
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T11:01:26.350+0000,2015-03-22T13:13:34.000+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T11:01:26.350+0000,2015-03-22T14:03:18.000+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T11:01:26.350+0000,2015-03-22T13:38:39.000+0000


In [0]:
%scala
//from_utc_timestamp()
val fromUtcTimestamp = toUtcTimestamp.withColumn("local", from_utc_timestamp($"UTC", "Asia/Singapore"))
display(fromUtcTimestamp)

timestamp,unix,Date,current_date,current_timestamp,UTC,local
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-22,2025-03-22T11:03:14.989+0000,2015-03-22T13:13:34.000+0000,2015-03-22T21:13:34.000+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-22,2025-03-22T11:03:14.989+0000,2015-03-22T14:03:18.000+0000,2015-03-22T22:03:18.000+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-22,2025-03-22T11:03:14.989+0000,2015-03-22T13:38:39.000+0000,2015-03-22T21:38:39.000+0000
